In [48]:
import pandas as pd
import os
import numpy as np

In [49]:
file_list = [f for f in os.listdir('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData') if f.endswith('.txt')]

In [50]:
delimiter = ','  # Change this to the actual delimiter if it's different
data_all = pd.DataFrame()
for file_path in file_list:
    # Read the text file into a Pandas DataFrame
    df = pd.read_csv('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData/' + file_path, delimiter=delimiter, header=None)
    df = df.iloc[:, :12]
    names = ['TimeStamp', 'Station', 'District', 'Freeway_#', 'Dir_Travel', 'Type', 'Length', 'Samples',
             'Observed_%', 'Total_Flow', 'Occupancy', 'Speed']
    df.columns = names
    df = df[df['Type']=="ML"]

    data_all = pd.concat([data_all, df], ignore_index=True)

In [51]:
data_all_observed = data_all[data_all['Observed_%'] > 0]
summary_observed = data_all_observed.groupby('Station').size().reset_index(name='Count')

In [52]:
metadata = pd.read_csv('Data_Raw/Archive/d07_text_meta_2022_07_15.txt', delimiter='\t')
metadata = metadata[metadata['Type']=="ML"]

In [53]:
metadata_obs = pd.merge(metadata, summary_observed, left_on='ID', right_on='Station' , how='left')

In [54]:
#metadata_obs.to_csv('Data_Raw/Station_MetaData.csv', index=False)

## Station Selection

In [55]:
station_selected = pd.read_csv('Data_Raw/CA23N_Exported.csv')

In [56]:
station_selected['PM_Dir'] = station_selected['Abs_PM'] - station_selected['Abs_PM'].min()

In [57]:
station_selected['Station_Number'] = station_selected['Abs_PM'].rank(ascending=True)
station_selected['Station_PM'] = station_selected['PM_Dir'].round(1) * 10

ID_to_remove = [771421]
station_selected = station_selected[~station_selected['ID'].isin(ID_to_remove)]

In [58]:
station_selected

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,...,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4,Station,Count,PM_Dir,Station_Number,Station_PM
0,771253,23,N,7,111,78582.0,R3.92,12.523,34.179522,-118.860080,...,HILLCREST,4850,NaN,NaN,NaN,771253.0,2016.0,0.00,1.0,0.0
1,771268,23,N,7,111,78582.0,R5.05,13.653,34.196191,-118.859225,...,JANSS 1,4745,NaN,NaN,NaN,NaN,NaN,1.13,3.0,11.0
2,771285,23,N,7,111,78582.0,R5.25,13.853,34.198898,-118.857485,...,JANSS 2,4747,NaN,NaN,NaN,771285.0,2016.0,1.33,4.0,13.0
3,771294,23,N,7,111,78582.0,R5.61,14.213,34.203343,-118.853680,...,LAS FLORES,2741,NaN,NaN,NaN,771294.0,2016.0,1.69,5.0,17.0
4,771306,23,N,7,111,78582.0,R6.02,14.623,34.208327,-118.849291,...,LOS ARBOLES 1,4749,NaN,NaN,NaN,NaN,NaN,2.10,6.0,21.0
5,771318,23,N,7,111,78582.0,R6.22,14.823,34.211348,-118.848860,...,LOS ARBOLES 2,4751,NaN,NaN,NaN,771318.0,2016.0,2.30,7.0,23.0
6,771329,23,N,7,111,78582.0,R7.13,15.733,34.225045,-118.847994,...,SUNSET HILLS 1,4753,NaN,NaN,NaN,NaN,NaN,3.21,9.0,32.0
7,771340,23,N,7,111,78582.0,R7.37,15.973,34.228473,-118.846374,...,SUNSET HILLS 2,4755,NaN,NaN,NaN,NaN,NaN,3.45,10.0,35.0
8,771349,23,N,7,111,78582.0,R6.47,15.073,34.215167,-118.849269,...,PEDERSON,2742,NaN,NaN,NaN,NaN,NaN,2.55,8.0,26.0
9,771360,23,N,7,111,78582.0,R8.3,16.903,34.241576,-118.839735,...,OLSEN 1,4757,NaN,NaN,NaN,771360.0,2016.0,4.38,11.0,44.0


In [59]:
order_match = station_selected[['ID', 'Station_PM']]

In [60]:
ID_selected = station_selected['ID']
data_all_select = data_all[data_all['Station'].isin(ID_selected)]
data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
data_all_select['TimeStep'] = 1 + (data_all_select['TimeDiff'] // 5)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_14650/440614197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_14650/440614197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_1

In [61]:
#station_selected[station_selected['PM_Dir_round'].duplicated(keep=False)]

In [62]:
data_all_select = pd.merge(data_all_select, order_match, left_on='Station', right_on='ID' , how='left')

In [63]:
Flow = data_all_select[['TimeStep', 'Station_PM', 'Total_Flow']]
Flow['TimeStep'] = Flow['TimeStep'].astype(int)
Flow['Station_PM'] = Flow['Station_PM'].astype(int)

Occupancy = data_all_select[['TimeStep', 'Station_PM', 'Occupancy']]
Occupancy['TimeStep'] = Occupancy['TimeStep'].astype(int)
Occupancy['Station_PM'] = Occupancy['Station_PM'].astype(int)

Speed = data_all_select[['TimeStep', 'Station_PM', 'Speed']]
Speed['TimeStep'] = Speed['TimeStep'].astype(int)
Speed['Station_PM'] = Speed['Station_PM'].astype(int)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_14650/3535391722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['TimeStep'] = Flow['TimeStep'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_14650/3535391722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_14650/3535391722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## Final Data Cleaning

In [64]:
"""
Create a table with timestep and PM (posted Miles) as one-one match
"""
timestep_value = [i for i in range(1, Occupancy['TimeStep'].max() + 1) for _ in range(Occupancy['Station_PM'].max() + 1)]
PM_values = list(range(Occupancy['Station_PM'].max() + 1)) * Occupancy['TimeStep'].max()
Time_PM = pd.DataFrame({'TimeStep': timestep_value, 'Station_PM': PM_values})

In [65]:
Occupancy_all = pd.merge(Time_PM, Occupancy, on=['TimeStep', 'Station_PM'] , how='left')
Flow_all = pd.merge(Time_PM, Flow, on=['TimeStep', 'Station_PM'] , how='left')
Speed_all = pd.merge(Time_PM, Speed, on=['TimeStep', 'Station_PM'] , how='left')

In [66]:
Occupancy_all

,TimeStep,Station_PM,Occupancy
0,1,0,0.0297
1,1,1,NaN
2,1,2,NaN
3,1,3,0.0554
4,1,4,NaN
...,...,...,...
153211,2016,71,NaN
153212,2016,72,NaN
153213,2016,73,NaN
153214,2016,74,NaN


In [67]:
Occupancy_all_wide = Occupancy_all.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Flow_all_wide = Flow_all.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Speed_all_wide = Speed_all.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [68]:
Flow_wide = Flow.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Occupancy_wide = Occupancy.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Speed_wide = Speed.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [69]:
Flow_all_wide.to_csv('Data_Clean/Flow_all.csv')
Occupancy_all_wide.to_csv('Data_Clean/Occupancy_all.csv')
Speed_all_wide.to_csv('Data_Clean/Speed_all.csv')

In [70]:
Flow_wide.to_csv('Data_Clean/Flow.csv')
Occupancy_wide.to_csv('Data_Clean/Occupancy.csv')
Speed_wide.to_csv('Data_Clean/Speed.csv')

In [71]:
Flow.to_csv('Data_Clean/Flow_Long.csv', index=False)
Occupancy.to_csv('Data_Clean/Occupancy_Long.csv', index=False)
Speed.to_csv('Data_Clean/Speed_Long.csv', index=False)

In [72]:
Flow_all.to_csv('Data_Clean/Flow_all_Long.csv', index=False)
Occupancy_all.to_csv('Data_Clean/Occupancy_all_Long.csv', index=False)
Speed_all.to_csv('Data_Clean/Speed_all_Long.csv', index=False)